<a href="https://colab.research.google.com/github/ahmedmabrouk24/Nueral-Network/blob/main/Neural_Network_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tflearn 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=94f720376b5702f8319b21e88c5e3983f33da730262f1634166abb1e43718c74
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install imgaug

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install pyyaml h5py  # Required to save models in HDF5 format

In [4]:
from random import shuffle
from tqdm import tqdm
import tensorflow as tf
import pandas as pd
from tensorflow.keras import layers,models,Sequential
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Dense, Conv2D, Activation, Flatten, Dropout, BatchNormalization, MaxPooling2D
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy
from keras.saving.save import load_model
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
import cv2
import random
import numpy as np
import os
import matplotlib.pyplot as plt
import imageio
import imgaug as ia
import imgaug.augmenters as iaa
from PIL import Image
import shutil
import json


In [5]:

TRAIN_DIR = '/kaggle/input/nn23-sports-image-classification/Train'
TEST_DIR = '/kaggle/input/nn23-sports-image-classification/Test'
train_images=[]
test_images=[]
val_images=[]
IMG_SIZE = 200
num_classes = 6
MODEL_NAME = 'sports_classification-cnn'


In [6]:
def augmentation(input_img):
        list_data=[]

    #Crop
        crop2 = iaa.Crop(percent=(0, 0.2)) 
        input_crop2 = crop2.augment_image(input_img)
        list_data.append(input_crop2)


        rot3 = iaa.Affine(rotate=(-40,20))
        input_rot3 = rot3.augment_image(input_img)
        list_data.append(input_rot3)
        
        blur=iaa.Sometimes(
        0.5,
        iaa.GaussianBlur(sigma=(0, 0.5)))
        input_blur=blur.augment_image(input_img)
        list_data.append(input_blur)

    #noise
   
        noise2=iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5)
        input_noise2=noise2.augment_image(input_img)
        list_data.append(input_noise2)
        
        aug=iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
        rotate=(-25, 25),
        shear=(-8, 8))
        input_aug=aug.augment_image(input_img)
        list_data.append(input_aug)
    

        return list_data

In [7]:
def create_label(image_name):
    img = image_name[0:4]
    if img == 'Bask':
        return 0
    elif img == 'Foot':
        return 1
    elif img == 'Rowi':
        return 2
    elif img == 'Swim':
        return 3
    elif img == 'Tenn':
        return 4
    elif img == 'Yoga':
        return 5

In [8]:
def create_train_data():
    training_data = []
    t = 0
    for img in tqdm(os.listdir(TRAIN_DIR)):
        if (t < 300):
          t+=1
          continue
        path = os.path.join(TRAIN_DIR, img)
        img_data = cv2.imread(path,1)
        img_data = cv2.cvtColor(img_data, cv2.COLOR_BGR2RGB)

        label_list = create_label(img)
        img_data = cv2.resize(img_data, (IMG_SIZE, IMG_SIZE))
        training_data.append([np.array(img_data), label_list])

        #append augmentation to train data
        aug_list = augmentation(img_data);
        for aug in aug_list:
            training_data.append([np.array(aug), label_list])

    print("Train size = " , len(training_data))
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data

In [9]:
column1=[]
def create_test_data():
    test_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
      
        path = os.path.join(TEST_DIR, img)
        img_data = cv2.imread(path,1)
        column1.append(img)
        img_data = cv2.cvtColor(img_data, cv2.COLOR_BGR2RGB)

        img_data = cv2.resize(img_data, (IMG_SIZE, IMG_SIZE))
        test_data.append([np.array(img_data), 0])

    np.save('test_data.npy', test_data)
    return test_data


In [10]:
def create_validation_data():
    validation_data = []
    t = 0
    for img in tqdm(os.listdir(TRAIN_DIR)):
        if (t > 300):
          break
        t+=1
        path = os.path.join(TRAIN_DIR, img)
        img_data = cv2.imread(path,1)
        img_data = cv2.cvtColor(img_data, cv2.COLOR_BGR2RGB)

        label_list = create_label(img)
        img_data = cv2.resize(img_data, (IMG_SIZE, IMG_SIZE))
        validation_data.append([np.array(img_data), label_list])

    shuffle(validation_data)
    np.save('validation_data.npy', validation_data)
    return validation_data

In [11]:
if (os.path.exists('train_data.npy')):
    all_images = np.load('train_data.npy',allow_pickle=True)
else: 
    all_images = create_train_data()

print("all images = ",len(all_images))
for i in range(0 , len(all_images)):
  train_images.append(all_images[i]);

X_train = np.array([i[0] for i in train_images]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
y_train = np.array([i[1] for i in train_images]).reshape(-1,)

#==============================================================================================

if (os.path.exists('validation_data.npy')):
    all_images_val = np.load('validation_data.npy',allow_pickle=True)
else: 
    all_images_val = create_validation_data()
print("all images = ",len(all_images_val))
for i in range(0 , len(all_images_val)):
  val_images.append(all_images_val[i]);

X_val = np.array([i[0] for i in val_images]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
Y_val = np.array([i[1] for i in val_images]).reshape(-1,)


#==============================================================================================

if (os.path.exists('test_data.npy')):
    all_images_test = np.load('test_data.npy',allow_pickle=True)
else: 
    all_images_test = create_test_data()
print("all images = ",len(all_images_test))
for i in range(0 , len(all_images_test)):
  test_images.append(all_images_test[i]);

X_test = np.array([i[0] for i in test_images]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
Y_test = np.array([i[1] for i in test_images]).reshape(-1,)


100%|██████████| 1681/1681 [00:29<00:00, 57.90it/s] 
/opt/conda/lib/python3.7/site-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


Train size =  8286
all images =  8286


 18%|█▊        | 301/1681 [00:02<00:11, 121.46it/s]


all images =  301


100%|██████████| 688/688 [00:09<00:00, 72.49it/s] 


all images =  688


In [18]:
model1 = Sequential([
        layers.Conv2D(filters = 32 ,kernel_size = (3, 3), input_shape = (IMG_SIZE, IMG_SIZE, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(filters = 32 ,kernel_size = (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size = (2,2)),

        layers.Conv2D(filters = 64 ,kernel_size = (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(filters = 64 ,kernel_size = (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2,2)),
    
        layers.Conv2D(filters = 128 ,kernel_size = (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2,2)),
    
        layers.Conv2D(filters = 256 ,kernel_size = (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2,2)),
    
        layers.Conv2D(filters = 512 ,kernel_size = (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Dropout(0.5),
    
        layers.Flatten(),
       
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(64, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(6, activation = 'softmax')
])

    
model1.compile(
    optimizer= tf.keras.optimizers.RMSprop(learning_rate = 0.000009),
    loss=SparseCategoricalCrossentropy(),
    metrics=['accuracy'])
   
if (os.path.exists('model.h5')):
      model1 = load_model('model.h5')
else : 
    model1.fit(X_train, y_train, validation_data = (X_val, Y_val), epochs = 40)
    #model1.save('model.h5')

Train on 8286 samples, validate on 301 samples
Epoch 1/40
8286/8286 [==============================] - ETA: 0s - loss: 1.6619 - acc: 0.4112

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


8286/8286 [==============================] - 22s 3ms/sample - loss: 1.6619 - acc: 0.4112 - val_loss: 1.1596 - val_acc: 0.5847
Epoch 2/40
8286/8286 [==============================] - 20s 2ms/sample - loss: 1.1662 - acc: 0.5932 - val_loss: 0.9235 - val_acc: 0.6877
Epoch 3/40
8286/8286 [==============================] - 21s 2ms/sample - loss: 0.9731 - acc: 0.6560 - val_loss: 0.8409 - val_acc: 0.7409
Epoch 4/40
8286/8286 [==============================] - 21s 2ms/sample - loss: 0.8513 - acc: 0.7118 - val_loss: 0.7913 - val_acc: 0.7741
Epoch 5/40
8286/8286 [==============================] - 20s 2ms/sample - loss: 0.7403 - acc: 0.7490 - val_loss: 0.7322 - val_acc: 0.7741
Epoch 6/40
8286/8286 [==============================] - 20s 2ms/sample - loss: 0.6886 - acc: 0.7683 - val_loss: 0.6822 - val_acc: 0.7841
Epoch 7/40
8286/8286 [==============================] - 21s 2ms/sample - loss: 0.5982 - acc: 0.7969 - val_loss: 0.6601 - val_acc: 0.7907
Epoch 8/40
8286/8286 [==============================

In [19]:
def test():
  Y_test=model1.predict(X_test)
  print(len(X_test))
  prediction=[]
  for i in Y_test:
    prediction.append(np.argmax(i))
  data={'image_name':column1,'label':prediction}
  print(len(column1),len(prediction))
  df=pd.DataFrame(data)
  df.to_csv('customBest.csv',index=False)
  print(df)
test()


688
688 688
    image_name  label
0      623.jpg      0
1      208.jpg      5
2      473.jpg      0
3      333.jpg      1
4      537.jpg      1
..         ...    ...
683    364.jpg      3
684     90.jpg      3
685    599.jpg      3
686     25.jpg      1
687    147.jpg      3

[688 rows x 2 columns]
